In [1]:
import os
import pandas as pd
import json
from utils.statistics import *
from utils.preprocess import *
import shutil

In [2]:
source_root = "/home/techt/Desktop/AI_for_Science/archiv/gold_icml"
dest_root = "/home/techt/Desktop/AI_for_Science/preprocessed/ours"  # specify

In [3]:
def org_normalize_trait(trait):
    trait = '-'.join(trait.split())
    normalized_trait = ''.join(trait.split("'"))
    return normalized_trait

In [4]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_nm_traits = [normalize_trait(at) for at in all_traits]
all_org_nm_traits = [org_normalize_trait(at) for at in all_traits]

In [5]:
for trait in os.listdir(source_root):
    if not os.path.isdir(os.path.join(source_root, trait)):
        continue
    if trait not in all_nm_traits:
        if trait in all_org_nm_traits:
            print('hhh')
            n_trait = all_nm_traits[all_org_nm_traits.index(trait)]
            os.rename(os.path.join(source_root, trait), os.path.join(source_root, n_trait))
            print(f"'{trait}' trait has been newly normalized to {n_trait} in the dir name")
        else:
            print('xxxxxxxxx', trait)


## check Xena data omission

In [7]:
s_ls = []
for t in os.listdir(dest_root):
    s_path = os.path.join(source_root, t)
    if not os.path.isdir(s_path):
        #f_ls = os.listdir(os.path.join(dest_root, t))
        #if len(f_ls) == 0 or (len(f_ls) == 1 and f_ls[0].endswith('.json')):
            #pass
        #else:
            # print(f"Strange. Dir {s_path} does not exist!")
        #s_ls.append(t)
        pass
    else:
        for f in os.listdir(s_path):
            if f.endswith('Xena.csv'):
                source_fpath = os.path.join(s_path, f)
                if os.path.isfile(os.path.join(dest_root, t, 'Xena.csv')):
                    print(f"{os.path.join(dest_root, t, 'Xena.csv')} exist corresponding to {source_fpath}")
                else:
                    print(f"No! {os.path.join(dest_root, t, 'Xena.csv')} not exist corresponding to {source_fpath}")
                    shutil.copyfile(os.path.join(source_root, t, 'Xena.csv'), os.path.join(dest_root, t, 'Xena.csv'))

/home/techt/Desktop/AI_for_Science/preprocessed/ours/Breast_Cancer/Xena.csv exist corresponding to /home/techt/Desktop/AI_for_Science/archiv/gold_icml/Breast_Cancer/Xena.csv
/home/techt/Desktop/AI_for_Science/preprocessed/ours/Bladder_Cancer/Xena.csv exist corresponding to /home/techt/Desktop/AI_for_Science/archiv/gold_icml/Bladder_Cancer/Xena.csv
/home/techt/Desktop/AI_for_Science/preprocessed/ours/Colon_and_Rectal_Cancer/Xena.csv exist corresponding to /home/techt/Desktop/AI_for_Science/archiv/gold_icml/Colon_and_Rectal_Cancer/Xena.csv


In [ ]:
sorted(s_ls)

## Fix preprocessed data
delete table with missing genetic data, and unify trait name as header. Creat json files 'cohort_info.json'.

In [ ]:
def org_normalize_trait(trait):
    trait = '-'.join(trait.split())
    normalized_trait = ''.join(trait.split("'"))
    return normalized_trait

In [ ]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_nm_traits = [normalize_trait(at) for at in all_traits]
all_org_nm_traits = [org_normalize_trait(at) for at in all_traits]

In [ ]:
sorted(all_nm_traits)

In [ ]:
len(set(all_nm_traits))

In [ ]:
sorted(all_org_nm_traits)

In [ ]:
# 检查路径和列名里的trait_name，进行normalize
# 没有gene的table去掉
# 创建或修改json文件

In [9]:
for trait in os.listdir(dest_root):
    if trait not in all_nm_traits:
        if trait in all_org_nm_traits:
            n_trait = all_nm_traits[all_org_nm_traits.index(trait)]
            os.rename(os.path.join(dest_root, trait), os.path.join(dest_root, n_trait))
            print(f"'{trait}' trait has been newly normalized to {n_trait} in the dir name")
            trait = n_trait
        else:
            print(f"!!!!!!!!!!!!!!!!!!'{trait}' strange trait")
            continue
    trait_dir = os.path.join(dest_root, trait)
    if not os.path.isdir(trait_dir):
        continue
    for fn in os.listdir(trait_dir):
        if fn.endswith('.log'):
            continue
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            df = pd.read_csv(fp)
            if 'Unnamed: 0' in df.columns:
                print(f"REDUNDANT COLUMN in '{fp}'")
                df = df.drop(columns = ['Unnamed: 0'], errors="ignore")
                df.to_csv(fp, index=False)
            header = df.columns[0]
            if header not in all_nm_traits:
                if header in all_traits:
                    nm_trait = all_nm_traits[all_traits.index(header)]
                    print(f"The header {header} should be replaced by {nm_trait} in '{fn}'")
                    df.columns = [nm_trait] + df.columns[1:].tolist()
                    df.to_csv(fp, index=False)
                    print('This cohort updated')
                else:
                    print(f"Strange header {header}")

            # elif header == "Lupus-(Systemic-Lupus-Erythematosus":
            #     df.columns = ["Lupus (Systemic Lupus Erythematosus)"] + df.columns[1:].tolist()
            #     df.to_csv(fp, index=False)
            #     print("The cohort for 'Lupus' updated")

            if len(df.columns) <= 4:
                print(f"NO GENE DATA in '{fp}'")
                os.remove(fp)
                print("File removed successfully")


!!!!!!!!!!!!!!!!!!'Congenital_Adrenal_Hyperplasia.log' strange trait
!!!!!!!!!!!!!!!!!!'Hepatitis.log' strange trait
!!!!!!!!!!!!!!!!!!'Atherosclerosis.log' strange trait
!!!!!!!!!!!!!!!!!!'Bladder_Cancer.log' strange trait
REDUNDANT COLUMN in '/home/techt/Desktop/AI_for_Science/preprocessed/ours/Breast_Cancer/GSE248830.csv'
!!!!!!!!!!!!!!!!!!'Height.log' strange trait
!!!!!!!!!!!!!!!!!!'Alopecia.log' strange trait
REDUNDANT COLUMN in '/home/techt/Desktop/AI_for_Science/preprocessed/ours/Hepatitis/GSE78160.csv'
!!!!!!!!!!!!!!!!!!'Congestive_heart_failure.log' strange trait
REDUNDANT COLUMN in '/home/techt/Desktop/AI_for_Science/preprocessed/ours/Aniridia/GSE204791.csv'
REDUNDANT COLUMN in '/home/techt/Desktop/AI_for_Science/preprocessed/ours/Aniridia/GSE137997.csv'
REDUNDANT COLUMN in '/home/techt/Desktop/AI_for_Science/preprocessed/ours/Aniridia/GSE137996.csv'
!!!!!!!!!!!!!!!!!!'Breast_Cancer.log' strange trait
REDUNDANT COLUMN in '/home/techt/Desktop/AI_for_Science/preprocessed/ours/

In [ ]:
dd = pd.read_csv("./trait_related_genes.csv")
for i in range(len(dd)):
    tr = dd.iat[i, 1]
    if tr in all_traits:
        dd.iat[i, 1] = normalize_trait(tr)
        print(f"{tr} normalized to {dd.iat[i, 1]}")
    else:
        assert tr in ['Age', 'Gender']

In [ ]:
dd.to_csv("./trait_related_genes.csv", index=False)

In [ ]:
de = pd.read_csv("./trait_condition_pairs.csv")
for i in range(len(de)):
    w = de.iat[i, 1]
    z = de.iat[i, 2]
    de.iat[i, 1] = normalize_trait(w)
    de.iat[i, 2] = normalize_trait(z)

In [ ]:
de.to_csv("./trait_condition_pairs.csv", index=False)

In [ ]:
de = pd.read_csv("./trait_condition_pairs.csv")
for i in range(len(de)):
    tr = de.iat[i, 1]
    if tr in all_traits:
        dd.iat[i, 1] = normalize_trait(tr)
        print(f"{tr} normalized to {dd.iat[i, 1]}")
    else:
        assert tr in ['Age', 'Gender']

In [13]:
for trait in os.listdir(dest_root):
    if trait not in all_nm_traits:
        if trait in all_traits:
            print(f"'{trait}' trait needs to be normalized ")
        else:
            print(f"'{trait}' strange trait")
        continue
    trait_dir = os.path.join(dest_root, trait)
    meta = dict()
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            cohort = fn[:-4]
            df = pd.read_csv(fp)
            header = df.columns[0]
            if header not in all_nm_traits:
                print(f'strange header {header}')
                continue
            trait_type = 'binary' if len(df[header].unique()) <= 2 else 'continuous'
            # May mistakenly judge (28, 28, 28, 28, ..., 100) as unbiased
            # May later try len(unique_values) == 2 and set(unique_values) <= {0, 1}
            trait_biased, _ = judge_and_remove_biased_features(df, header, trait_type)
            has_age = 'Age' in df.columns or 'age' in df.columns
            has_gender = 'Gender' in df.columns or 'gender' in df.columns
            sample_size = len(df)
            if not trait_biased:
                meta[cohort] = {"is_usable": True, "is_available": True, "is_biased": False, "has_age": has_age, "has_gender": has_gender, "sample_size": sample_size, "note": ""}
    js_path = os.path.join(trait_dir, "cohort_info.json")
    if os.path.exists(js_path):
        os.remove(js_path)
    with open(js_path, 'w') as file:
        json.dump(meta, file)


'Congenital_Adrenal_Hyperplasia.log' strange trait
'Hepatitis.log' strange trait
'Atherosclerosis.log' strange trait
'Bladder_Cancer.log' strange trait
For the feature 'Breast_Cancer', the least common label is '0.0' with 2 occurrences. This represents 10.53% of the dataset.
The distribution of the feature 'Breast_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 46.0
  50% (Median): 49.0
  75%: 54.0
Min: 40.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Breast_Cancer', the least common label is '0' with 113 occurrences. This represents 9.30% of the dataset.
The distribution of the feature 'Breast_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gen

In [ ]:
import pandas as pd
import ast
from utils.preprocess import normalize_gene_symbols

def normalize_and_save_genes(file_path, output_file_path):
    """Read a CSV file about gene-trait association, normalize the related genes for each trait, remove None values,
    and save the modified table to a new file."""
    # Read the CSV file into a DataFrame
    related_gene_df = pd.read_csv(file_path)
    related_gene_df = related_gene_df.loc[:, ['Trait', 'Related_Genes']].set_index('Trait')

    # Normalize gene symbols and remove None values
    def normalize_genes(genes):
        # Convert the string representation of the list to a list
        gene_list = ast.literal_eval(genes)
        # Normalize gene symbols using the provided function
        normalized_genes = normalize_gene_symbols(gene_list)
        # Filter out None values
        return [gene for gene in normalized_genes if gene is not None]

    # Apply normalization to each row in the 'Related_Genes' column
    related_gene_df['Related_Genes'] = related_gene_df['Related_Genes'].apply(normalize_genes)

    # Save the modified DataFrame to a new CSV file
    related_gene_df.to_csv(output_file_path)

# Example usage:
# normalize_and_save_genes('./trait_related_genes_org.csv', './trait_related_genes.csv')

In [ ]:
normalize_and_save_genes('./trait_related_genes_org.csv', './trait_related_genes.csv')

## Clean GEO data

### Normalize trait names

In [ ]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_nm_traits = [normalize_trait(at) for at in all_traits]
all_org_nm_traits = [org_normalize_trait(at) for at in all_traits]

In [ ]:
len(all_nm_traits)

In [ ]:
geo_root = "/media/techt/DATA/GEO"

In [ ]:
strange = []
for trait in os.listdir(geo_root):
    if trait in all_org_nm_traits:
        n_trait = all_nm_traits[all_org_nm_traits.index(trait)]
        if n_trait != trait:
            #print(f"'{trait}' trait should be newly normalized to '{n_trait}' in the dir name")
            os.rename(os.path.join(geo_root, trait), os.path.join(geo_root, n_trait))
    else:
        strange.append(trait)
        print(f"!!!!!!!!!!!!!!!!!!'{trait}' strange trait")
        continue

In [ ]:
strange

In [ ]:
sum(sizes[:20])

In [ ]:
get_gene_annotation(os.path.join(directory_path, 'Retinoblastoma/GSE51334/GSE51334_family.soft.gz'))

In [ ]:
files[:30]

In [ ]:
import gzip

In [ ]:
fp = os.path.join(directory_path, 'Mitochondrial_Disorders/GSE25925/GSE25925_family.soft.gz')

In [ ]:
def get_genetic_data(file_path, marker="!series_matrix_table_begin"):
    """Read the gene expression data into a dataframe, and adjust its format"""
    # Determine the number of rows to skip
    with gzip.open(file_path, 'rt') as file:
        for i, line in enumerate(file):
            if marker in line:
                skip_rows = i + 1  # +1 to skip the marker row itself
                print(skip_rows)
                break
        else:
            raise ValueError(f"Marker '{marker}' not found in the file.")

    # Read the genetic data into a dataframe
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=skip_rows, comment='!', delimiter='\t')
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'}).astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data

In [ ]:
get_genetic_data(fp)

In [ ]:
genetic_data = get_genetic_data(os.path.join(directory_path, 'Atrial_Fibrillation/GSE213029/GSE213029_family.soft.gz'), compression='gzip', skiprows=52, comment='!', delimiter='\t')

In [ ]:
def get_genetic_data(file_path, marker="!series_matrix_table_begin"):
    """Read the gene expression data into a dataframe, and adjust its format"""
    # Determine the number of rows to skip
    with gzip.open(file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if marker in line:
                skip_rows = i + 1  # +1 to skip the marker row itself
                break
        else:
            raise ValueError(f"Marker '{marker}' not found in the file.")

    # Read the genetic data into a dataframe
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=skip_rows, comment='!', delimiter='\t')
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'}).astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data